# Semi-supervised BERTopic Modeling with Probability Calculation
https://maartengr.github.io/BERTopic/getting_started/semisupervised/semisupervised.html

## Setup

In [1]:
import numba
import warnings
warnings.filterwarnings("ignore", category=numba.NumbaDeprecationWarning)

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

import csv
import re

from nltk.corpus import stopwords

from hdbscan import HDBSCAN

import pandas as pd

import numpy as np

#from sentence_transformers import SentenceTransformer

/Users/aiolf1/anaconda3/envs/BERTopic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import and Clean Data

In [2]:
df = pd.read_csv('data/all_businesses.tsv', sep='\t', dtype={'BusinessShortNumber': str})

# Split text and TagNames
docs = df[["BusinessShortNumber", "all"]]
categories = df["TagNames"]

# Convert categories to numbers
categories = pd.factorize(categories)[0]

In [3]:
# Remove punctuation
docs['all'] = docs['all'].apply(lambda doc: re.sub(r'[^\w\s]', '', doc))


# Lowercase
docs['all'] = docs['all'].apply(lambda doc: doc.lower())


# Remove stopwords
german_stop_words = stopwords.words('german')

# Import custom stopwords file as list of strings
with open('../../data/custom_stopwords.txt', 'r') as f:
   custom_stopwords = f.readlines()

# remove whitespace characters like `\n` at the end of each line
custom_stopwords = [x.strip() for x in custom_stopwords]

# remove stopwords from docs
docs['all'] = docs['all'].apply(lambda doc: ' '.join(word for word in doc.split() if word not in german_stop_words))
docs['all'] = docs['all'].apply(lambda doc: ' '.join(word for word in doc.split() if word not in custom_stopwords))

# remove "na" from docs
docs = docs[docs['all'] != 'na']


## Modelling

In [4]:
# https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#hdbscan
hdbscan_model = HDBSCAN(
    min_cluster_size = 2,
    min_samples = 1,
    metric = 'euclidean',
    prediction_data = True)

# BERTopic German model
# Parameter tuning: https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#bertopic
topic_model = BERTopic(
    language = "multilingual",
    min_topic_size = 2, # minimum number of documents in a topic; 2-3 leads to around 180 topics
    nr_topics = 100, # number of topics; theyworkforyou.com have around 90-100 topics for each MP
    verbose = True,
    top_n_words = 10,
    n_gram_range = (1, 3),
    hdbscan_model = hdbscan_model,
    calculate_probabilities = True,
     # https://www.sbert.net/docs/pretrained_models.html
     # Probably saved somewhere here: /Users/aiolf1/anaconda3/envs/BERTopic/lib/python3.11/site-packages
    embedding_model = "paraphrase-multilingual-mpnet-base-v2").fit(docs['all'], y = categories) # perform supervised topic modeling, we simply use all categories
    #embedding_model = SentenceTransformer("intfloat/multilingual-e5-base")).fit(docs, y = categories) # perform supervised topic modeling, we simply use all categories

topics, probs = topic_model.fit_transform(docs['all'])

Batches: 100%|██████████| 21/21 [03:08<00:00,  8.99s/it]
2023-08-27 16:55:02,198 - BERTopic - Transformed documents to Embeddings
2023-08-27 16:55:10,961 - BERTopic - Reduced dimensionality
2023-08-27 16:55:11,372 - BERTopic - Clustered reduced embeddings
2023-08-27 16:55:14,564 - BERTopic - Reduced number of topics from 168 to 140
Batches: 100%|██████████| 21/21 [03:14<00:00,  9.24s/it]
2023-08-27 16:58:29,596 - BERTopic - Transformed documents to Embeddings
2023-08-27 16:58:33,378 - BERTopic - Reduced dimensionality
2023-08-27 16:58:33,910 - BERTopic - Clustered reduced embeddings
2023-08-27 16:58:37,707 - BERTopic - Reduced number of topics from 179 to 140


In [5]:
# Get Model Information
doc_info = topic_model.get_document_info(docs['all'])
topic_info = topic_model.get_topic_info()

In [6]:
# Merge model output with docs in order to preserve BusinessShortNumber
doc_info = pd.merge(doc_info, docs, left_on='Document', right_on='all', how='left')

# Add BusinessShortNumber to probs for export
probs_df = pd.DataFrame(probs)
probs_df["BusinessShortNumber"] = doc_info["BusinessShortNumber"]

# Remove duplicates (for testing purposes)
doc_info = doc_info.drop_duplicates(subset='BusinessShortNumber')
probs_df = probs_df.drop_duplicates(subset='BusinessShortNumber')

In [7]:
# Number of topics
num_topics = topic_info.shape[0]
print(f"There are {num_topics} topics.")

There are 140 topics.


## Save and Load Model

In [8]:
# Save model
topic_model.save("BERT_data/v230827/topic_model", serialization="safetensors", save_ctfidf=True)

# Save topics and probs to file
np.save('BERT_data/v230827/probs.npy', probs)
np.save('BERT_data/v230827/topics.npy', topics)

# Save ndarray as csv file
probs_df.to_csv('BERT_data/v230827/probs.csv', index=False)

# Save document level information to csv
doc_info.to_csv('BERT_data/v230827/doc_info.csv', index=False)

# Save topic information to csv
topic_info.to_csv('BERT_data/v230827/topic_info.csv', index=False)

In [ ]:
# Load model
topic_model = BERTopic.load("BERT_data/v230820/topic_model")

# Load topics and probs
probs = np.load('BERT_data/v230820/probs.npy', allow_pickle=True)
topics = np.load('BERT_data/v230820/topics.npy', allow_pickle=True)
topics = topics.tolist()